In [ ]:
import numpy as np
import scipy as sp
from scipy import signal
import matplotlib.pyplot as plt
import QPP_Funcs as qpp
import celerite as ce
import os

s0 = np.linspace(1,9.99,4)
q = np.linspace(0,2.99,5)

for s0i in s0:
    for qi in q:
        print("Running Q: "+ str(qi) + "\t S: " + str(s0i) +"\n")
        realparams = [-.13, -1.4] 
        qpoparams = [s0i, qi, .3]
        modelparams = [11.33844804, 6.92311406, 6.85207764]
        trueparams = realparams + qpoparams + modelparams

        model = qpp.CTSModel_prior(log_A = modelparams[0], log_tau1 = modelparams[1], log_tau2 = modelparams[2])
        kernel1 = qpp.SHOTerm_Prior(log_S0 = qpoparams[0], log_Q = qpoparams[1], log_omega0 = qpoparams[2])
        kernel2 = qpp.RealTerm_Prior(log_a = realparams[0], log_c = realparams[1])
        kernel = kernel2 + kernel1

        t = np.linspace(0,4000,2000)
        I = qpp.simulate(t, model, kernel, noisy = True)




        A_guess, t1_guess, t2_guess = qpp.initguess(t,I)
        model = qpp.CTSModel_prior(log_A = np.log(A_guess), log_tau1 = np.log(t1_guess), log_tau2 = np.log(t2_guess))
        kernel1 = qpp.SHOTerm_Prior(log_S0 = 0, log_Q = 0, log_omega0 = 0) #write guesser for kernel parameters
        kernel2 = qpp.RealTerm_Prior(log_a=0., log_c=0.) #write guesser for kernel parameters
        kernel = kernel2 + kernel1
        gp = ce.GP(kernel, mean=model, fit_mean=True)
        gp.compute(t, np.sqrt(I))
        initparams = gp.get_parameter_vector()



        soln = qpp.optimize_gp(gp, I)
        gp.set_parameter_vector(soln.x)
        figopt = qpp.plot_gp(t, I, np.sqrt(I), gp, model, predict=True, label = "Optimized fit")

        #sampling
        sampler = qpp.sample_gp(initparams, I, gp, burnin=8000, nwalkers=100, nsteps = 2000)
        chain = sampler.chain


        figsam = qpp.plot_chain(chain, labels = gp.get_parameter_names())


        figoptsam = qpp.plot_gp(t, I, np.sqrt(I), gp, model, chain=chain)

        figcorner, maxparams = qpp.plot_corner(chain, labels = gp.get_parameter_names(), truevals = trueparams)


        header = "Rednoise Kernel Params: \t"+str(realparams) + "\nQPO Params: \t" + str(qpoparams) + "\nEnvelope Params \t" + str(modelparams) + "\nTrue Parameter Vector: \t" + str(trueparams) + "\nMaxParams: \t" + str(maxparams)
        qpolabel = "A{:1.2f}".format(qpoparams[0])+"Q{:1.2f}".format(qpoparams[1])+"w{:1.2f}".format(qpoparams[2])
        fname = "Data/Simulating/simulated_burst_" + qpolabel+"/"

        if not os.path.exists(fname):
            os.makedirs(fname)
            qpp.store_flare(fname+"data", header, t, I, soln.x, chain)
            figsam.savefig(fname+"chain_ts.png")
            figoptsam.savefig(fname+"lc_plot.png")
            figcorner.savefig(fname+"chain_corner.png")

            textheader = open(fname+"header.txt",'w')
            textheader.write(header)
            textheader.close()
        else:
            print "directory exists"


Running Q: 0.0	 S: 1.0

Initial log-likelihood: -74969734.6072
Final log-likelihood: -13231.061308
Optimized log-parameters: [ 2.08525822e-05 -1.27240342e-05  1.59127701e-05 -1.91459327e-06
 -8.76715860e-07  1.13384610e+01  6.92337331e+00  6.85180036e+00]
Burning in...
Sampling...
Done!


Stored flare at Data/Simulating/simulated_burst_A1.00Q0.00w0.30/data
Running Q: 0.7475	 S: 1.0

Initial log-likelihood: -75234655.3002
Final log-likelihood: -13278.1444874
Optimized log-parameters: [ 2.52153153e-05 -1.23717716e-05  2.00883444e-05  2.81477010e-06
  5.76713637e-06  1.13385004e+01  6.92283268e+00  6.85223925e+00]
Burning in...
Sampling...
Done!
Stored flare at Data/Simulating/simulated_burst_A1.00Q0.75w0.30/data
Running Q: 1.495	 S: 1.0

Initial log-likelihood: -74369249.5642
Final log-likelihood: -13217.7439884
Optimized log-parameters: [ 2.55941072e-05 -1.31316288e-05  2.00039424e-05  1.09514053e-06
  3.83922150e-06  1.13386033e+01  6.92241444e+00  6.85234765e+00]
Burning in...
